In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs
from datetime import datetime
from dateutil import parser

In [2]:
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
#     }
# url = f"https://www.tempo.co/indeks/2023-11-01/nasional"
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
# body = soup.find('main', {"class": "main-left"})
# articles= body.findAll('h2', {"class": "title"})
# link=[]
# for article in articles:
#     link = article.find('a')['href']
#     print(link)
# links= cek.find_all('a')
# all_link=[]
# for link in links:
#         if link and 'href' in link.attrs:
#             link_href = link['href']
#             all_link.append(link_href)


https://nasional.tempo.co/read/1791540/politikus-golkar-sebut-usulan-hak-angket-masinton-hanya-gimik-politik-untuk-degradasi-prabowo-gibran
https://nasional.tempo.co/read/1791532/ganjar-mahfud-bentuk-tim-pemenangan-muda-diketuai-pengusaha-dharmaji-suradika
https://nasional.tempo.co/read/1791530/resmikan-rumah-sakit-dr-sadjiman-bogor-prabowo-nama-dokter-yang-selamatkan-nyawa-saya
https://nasional.tempo.co/read/1791519/ketua-mkmk-jimly-asshiddiqie-bilang-putusan-mk-bisa-dibatalkan-ini-syaratnya
https://nasional.tempo.co/read/1791509/ketua-prabowo-mania-08-tantang-ganjar-ungkap-aktor-pelanggaran-ham-27-juli-1996
https://nasional.tempo.co/read/1791503/jimly-asshiddiqie-sebut-pengesahan-revisi-pkpu-tak-pengaruhi-putusan-mkmk
https://nasional.tempo.co/read/1791497/wamendes-dilaporkan-ke-bawaslu-setelah-galang-dukungan-untuk-prabowo-gibran
https://nasional.tempo.co/read/1791478/johnny-g-plate-merasa-dizalimi-dan-difitnah-dalam-kasus-korupsi-bts-kominfo
https://nasional.tempo.co/read/1791471/j

In [3]:
def scrape_link_per_day(date):
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
    url = f"https://www.tempo.co/indeks/{date}/nasional"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.find('main', {"class": "main-left"})
    articles= body.findAll('h2', {"class": "title"})
    page_links=[]
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    print(f"Scraped {len(page_links)} links when date {date}")
    return page_links

In [4]:
all_link=scrape_link_per_day('2023-11-01')

Scraped 72 links when date 2023-11-01


In [5]:
def scrape_url(url,max_retries=3):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Extract data from the web page
                    title_elem = soup.find('h1', {"class": "title margin-bottom-sm"})
                    title_text = title_elem.text.strip() if title_elem else "Title not found"

                    date_elem = soup.find('p', {"class": "date margin-bottom-sm"})
                    date_text = date_elem.text.strip() if date_elem else "Date not found"
                    
                    date_part = ' '.join(date_text.split(',')[1:]).strip()
                    date_object = parser.parse(date_part)
                    formatted_date = date_object.strftime('%Y-%m-%d')

                    body_elem = soup.find('div', {"class": "detail-konten"})
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = "\n".join(p.text.strip() for p in content_elem)
                        content_text = content_text.replace('\n', ' ')
                    else:
                        content_text = "Content not found"

                    return {
                        'title': title_text,
                        'date': formatted_date,
                        'content': content_text,
                        'link': url
                    }
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [6]:
url='https://nasional.tempo.co/read/1791540/politikus-golkar-sebut-usulan-hak-angket-masinton-hanya-gimik-politik-untuk-degradasi-prabowo-gibran'
cek_hasil=scrape_url(url)

In [7]:
print(cek_hasil)

{'title': 'Politikus Golkar Sebut Usulan Hak Angket Masinton Hanya Gimik Politik untuk Degradasi Prabowo-Gibran', 'date': '2023-11-01', 'content': 'TEMPO.CO, Jakarta - Ketua Badan Pemenangan Pemilu Partai Golkar Maman Abdurahman menanggapi usulan politikus PDIP Masinton Pasaribu untuk mengusulkan hak angket DPR terhadap Mahkamah Konstitusi atau MK. Menurut Maman, usulan Masinton itu tak akan berimplikasi apa pun terhadap putusan MK soal batas usia capres-cawapres. "Saya pikir, terus kalau pun itu terwujud ada pengusulan hak angket, terus implikasinya juga apa? Kan nggak ada juga," kata dia saat ditemui di Kompleks Stadion Gelora Bung Karno, Jakarta Pusat, Rabu, 1 November 2023. Baca Juga: UM Surabaya Siap Gelar Uji Publik Pasangan Prabowo-Gibran, Ini Daftar Panelisnya Sebelumnya, dalam Rapat Paripurna Masa Sidang ke-8 DPR RI, Senin, 31 Oktober 2023, Masinton menyampaikan pendapatnya agar Dewan mengajukan hak angket untuk putusan MK soal batas usia capres-cawapres. Menurut Maman, apa ya